<h2>Pustaka dan utility function</h2>

In [1]:
import cv2
import threading
import time
from pyardrone import ARDrone, at
from time import sleep, perf_counter
import numpy as np
import queue
import os

# Fungsi untuk menghubungkan ke drone
def connect(drone, ip, navdemo):
    try:
        if drone is None:
            drone = ARDrone(host=ip)
            drone.send(at.CONFIG('general:navdata_demo', True))
            print("connected")
            connected = True
            battery = get_battery(drone,navdemo)
        else:
            if drone.state.navdata_bootstrap is not True:
                navdemo = drone.navdata.demo
                print("connected again")
                connected = True
                battery = get_battery(drone,navdemo)

            else:
                print("navdata demo is problem")
                drone.send(at.CONFIG('general:navdata_demo', True))
                sleep(1.5)
                navdemo = drone.navdata.demo
                connected = False
        sleep(0.5)
        return drone, navdemo, connected
    except Exception as e:
        print(f"Error during connection: {e}")
        return None, None, False

# Fungsi untuk memutuskan koneksi dari drone
def disconnect(drone):
    try:
        battery = get_battery(drone,navdemo)
        drone.close()
        drone = None
        return drone
    except Exception as e:
        print(f"Error during disconnection: {e}")

def get_battery(drone,navdemo):
    global battery
    if drone is not None:
        if drone.navdata_ready.is_set():
            if navdemo is not None:
                battery = navdemo.vbat_flying_percentage
            else:
                battery = "drone Not connected, cannot get battery status"
    else:
        battery = "drone Not connected, cannot get battery status"
    print(battery)
    return battery

def Vupdate(forward, backward,left,right):
    global Vforward
    global Vbackward
    global Vleft
    global Vright
    Vforward = forward
    Vbackward = backward
    Vleft = left
    Vright = right

# Fungsi untuk menggerakkan drone
def move(drone, forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0):
    try:
        if drone is not None:
            drone.move(forward=forward, backward=backward, right=right, left=left, up=up, down=down, cw=cw, ccw=ccw)
            Vupdate(forward, backward,left,right)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during drone movement: {e}")

# Fungsi untuk hover
def hover(drone):
    try:
        if drone is not None:
            drone.move(forward=0.0, backward=0.0, right=0.0, left=0.0, up=0.0, down=0.0, cw=0.0, ccw=0.0)
            Vupdate(0,0,0,0)
            print('drone berhasil hover')
            sleep(1)
        else:
            print("Drone belum terhubung")
    except Exception as e:
        print(f"Error during hover: {e}")

# Fungsi untuk takeoff
def takeoff(drone):
    try:
        print('takeoff')
        drone.takeoff()
        sleep(2)
        drone.send(at.CALIB(0))
        Vupdate(0,0,0,0)
    except Exception as e:
        print(f"Error during takeoff: {e}")

# Fungsi untuk landing
def land(drone):
    try:
        print('landing')
        drone.land()
        Vupdate(0,0,0,0)
    except Exception as e:
        print(f"Error during landing: {e}")

# Fungsi untuk mengganti kamera
def change_camera(drone, camera_condition):
    try:
        sending_state = True
        drone.send(at.CONFIG("video:video_channel", camera_condition))
        sending_state = False
        return camera_condition
    except Exception as e:
        print(f"Error during camera change: {e}")

def movAvg(window,dataMovAvgX,dataMovAvgY,xEst,yEst):
    if len(dataMovAvgX) >= (window-1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = (sum(dataMovAvgX)/len(dataMovAvgX))
        yAvg = (sum(dataMovAvgY)/len(dataMovAvgY))
        dataMovAvgX.pop(0)
        dataMovAvgY.pop(0)
    elif len(dataMovAvgX) < (window-1):
        dataMovAvgX.append(xEst)
        dataMovAvgY.append(yEst)
        xAvg = xEst
        yAvg = yEst
    return xAvg, yAvg,dataMovAvgX,dataMovAvgY

def saveImage(output_folder,file_counter,frame):
    # Simpan gambar ke file
    filename = os.path.join(output_folder, f'detected_marker_{file_counter:04d}.png')
    cv2.imwrite(filename, frame)
    file_counter += 1
    return file_counter
    # kalo mau pake fungsi, copas aja gini -> file_counter = saveImage(output_folder,file_counter,frame)

def dataAruco(num_rows,num_cols,jarak):
        
    total_markers = num_rows * num_cols
    databasePos = []
    dataID = []
    i = 1
    for row in range(num_cols):
        for col in range(num_rows):
            posX = jarak*row
            posY = jarak*col
            databasePos.append([posX, posY])
            dataID.append(i)
            i += 1
    # print(np.array(databasePos))
    # print(databasePos[dataID.index(500)])
    return dataID, databasePos

def convertPos(image, ids, corners, dataID, databasePos):

    ###### est posisi berdasarkan database posisi real #####
    sumx = 0
    sumy = 0
    for i in range(len(ids)): 
        sumx += databasePos[dataID.index(ids[i])][0]
        sumy += databasePos[dataID.index(ids[i])][1]

    x = sumx/len(ids)
    y = sumy/len(ids)

    ###### titik tengah setiap marker aruco #####
    tengah = []
    
    for i in range(len(ids)):
        xTengah = (int(corners[i][0][0][0])+int(corners[i][0][1][0])+int(corners[i][0][2][0])+int(corners[i][0][3][0]))/4
        yTengah = (int(corners[i][0][0][1])+int(corners[i][0][1][1])+int(corners[i][0][2][1])+int(corners[i][0][3][1]))/4
        tengah.append([xTengah, yTengah])

    ###### titik tengah rata rata (est posisi dalam pixel) #####
    sumtengahx = 0
    sumtengahy = 0
    for i in range(len(ids)):
        sumtengahx += tengah[i][0]
        sumtengahy += tengah[i][1]

    posPixelX = int(sumtengahx/len(ids))
    posPixelY = int(sumtengahy/len(ids))

    ###### Pusat Kamera #####
    height, width = image.shape[:2]
    pusatKameraX = int(width/2)
    pusatKameraY = int(height/2)

    ###### est posisi pusat kamera #####
    aX = 0.123
    aY = 0.131
    rXPixel = pusatKameraX - posPixelX
    rYPixel = pusatKameraY - posPixelY
    rX = rXPixel * aX
    rY = rYPixel * aY
    xEst = x + rX
    yEst = y + rY

    return x,y,posPixelX,posPixelY,xEst,yEst,pusatKameraX,pusatKameraY,tengah

def cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah):
    uniqDisplay = False
    if ids is not None:
        #Gambar deteksi marker pada frame
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        if uniqDisplay == True:
            radius = 5
            #Gambar titik pusat setiap marker aruco yang terdeteksi
            for i in range(len(ids)):
                xTengah = tengah[i][0]
                yTengah = tengah[i][1]
                cv2.circle(frame,(int(xTengah),int(yTengah)), int(radius) , (0,0,255), -1)
        
            # Gambar titik estimasi MFP
            cv2.circle(frame,(posPixelX,posPixelY), int(2*radius), (0,255,0), -1)

            # Gambar titik Pusat kamera
            cv2.circle(frame,(pusatKameraX,pusatKameraY), int(2*radius), (255,0,0), -1)

    # Tampilkan frame dengan deteksi marker
    cv2.imshow('Detected ArUco Markers', frame)

    return

def move_to_target(drone, xAvg, yAvg, target_x, target_y, first, error_x1, error_y1):
    """
    Fungsi untuk menggerakkan drone menuju target_x dan target_y berdasarkan posisi estimasi xAvg dan yAvg.
    """
    error_x = target_x - xAvg
    error_y = target_y - yAvg

    # print("error x = "+str(error_x)+" , error y = "+str(error_y))

    if first == True :
        error_x1 = error_x
        error_y1 = error_y
        first = False

    deltaErrorX = error_x-error_x1
    deltaErrorY = error_y-error_y1

    kec = 0.05
    kp = 0.5
    if (error_x > 0) and (error_x1 >= 0):
        right = kec
        left = 0
    elif (error_x > 0) and (error_x1 < 0):
        right = 0
        left = 0
    elif error_x == 0 :
        right = 0
        left = 0
    elif (error_x < 0) and (error_x1 <= 0) :
        right = 0
        left = kec
    elif (error_x < 0) and (error_x1 > 0) :
        right = 0
        left = 0

    if (error_y > 0) and (error_y1 >= 0):
        forward = kec
        backward = 0
    elif (error_y > 0) and (error_y1 < 0):
        forward = 0
        backward = 0
    elif error_y == 0 :
        forward = 0
        backward = 0
    elif (error_y < 0) and (error_y1 <= 0) :
        forward = 0
        backward = kec
    elif (error_y < 0) and (error_y1 > 0) :
        forward = 0
        backward = 0
    # print("command : Forward=" + str(forward) + " , Backward=" + str(backward) + " , right=" + str(forward) + " , left=" + str(left))
    move(drone, forward=forward, backward=backward, right=right, left=left)

    error_x1 = error_x
    error_y1 = error_y
    return first, error_x1, error_y1



<h2>Inisialisasi Variabel</h2>

In [24]:
drone = None
battery = None
vx = None
vy = None
vz = None
yaw = None
pitch = None
roll = None
altitude = None
flying_state = None
navdemo = None
camera_condition = "2"
sending_state = False
current_time = 0

ip = '192.168.1.1'


# Fungsi untuk menyimpan Navdata ke dalam DataFrame
navdata_records = [['current_time', 'battery', 'flying_state', 'altitude', 'vx', 'vy', 'vz', 'yaw', 'pitch', 'roll']]


<h2>Drone Land</h2>

In [17]:
drone = land(drone)

landing


<h2>Drone Connect</h2>

In [26]:
drone, navdemo, connected = connect(drone, ip, navdemo)

connected again
100


<h2>Drone Camera Set</h2>

In [27]:
camera_condition = change_camera(drone, 3)

<h2>Drone Disconnect</h2>

In [28]:
drone = disconnect(drone)

100


<h2>Integrasi 1 Loop</h2>

In [2]:
def udpConnect(video_ready_event,stop_event,):
    global drone
    global navdemo
    global connected
    global ip
    global moveCommand
    try:
        # Tunggu hingga video stream siap
        video_ready_event.wait()
        print("video nyala, bisa lanjut")
        drone, navdemo, connected = connect(drone, ip, navdemo)
        if drone is not None:
            if connected == True:
                print("drone konek")
                drone.navdata_ready.wait()  # wait until NavData is ready
                moveCommand = True
                drone_connect_event.set()
                return
            else :
                print("tidak dapat konek")
        else:
            print("drone tidak terdeteksi, coba periksa sambungan wifi")

    except Exception as e:
        print(f"Error comunicating via UDP: {e}")
    

######## Program Utama ############
if __name__ == "__main__":

    ######## Inisialisasi variabel ############

    # drone things
    ip = '192.168.1.1'
    drone = None
    battery = None
    vx = None
    vy = None
    vz = None
    yaw = None
    pitch = None
    roll = None
    altitude = None
    flying_state = None
    navdemo = None
    camera_condition = "2"
    sending_state = False
    connected = False

    # move command things
    
    kec = 0.1
    waktu_move = 1
    setup_udp = False
    setup_moving = False
    direction = 0
    moveCommand = False
    target_x = 200
    target_y = 250
    stopping = False
    loopWait = 250
    loop = 0
    Vforward = 0
    Vbackward = 0
    Vleft = 0
    Vright = 0

    # timing things
    init_time = perf_counter()
    last_time_move = init_time
    last_time_command = init_time
    last_time_input = init_time
    current_time = perf_counter()
    waktu_input = 0.015
    waktu_command_move = 0.033
    # Aruco things
    id_min1 = []
    ids = []
    tengah = []
    num_rows = 25
    num_cols = 20
    id_1 = []
    window = 25
    dataMovAvgX = []
    dataMovAvgY = []
    posPixelX = None
    posPixelY = None
    pusatKameraX = None
    pusatKameraY = None
    file_counter = 0
    kameraCheck = False

    # positon and control things
    posisiX = [None]
    posisiY = [None]
    first = True
    error_x1 = 0
    error_y1 = 0

    # debug & logging things
    plotX = []
    plotY = []
    dataAkhir = [["Waktu", "X_movAvg", "Y_movAvg", "V_forward", "V_backward", "V_left", "V_right" ]]
    navdata_records = [['current_time , battery , flying_state , altitude , vx , vy , vz , yaw , pitch , roll']]

    ######## Setup Multi Thread ############

    # Event untuk sinkronisasi
    video_ready_event = threading.Event()
    drone_connect_event = threading.Event()
    stop_event = threading.Event()  
     
    # Thread untuk memberikan command pergerakan kepada drone
    udpConnect_thread = threading.Thread(target=udpConnect, args=(video_ready_event,stop_event,))
    udpConnect_thread.start()

    try:

        ######## Setup kamera dan aruco detector ############
        ardrone_url = 'tcp://192.168.1.1:5555'
        camera = cv2.VideoCapture(ardrone_url)
        
        if not camera.isOpened():
            print("Gagal membuka stream video. Pastikan URL benar dan drone terhubung.")
            stop_event.set()
            # return

        dataID, databasePos = dataAruco(num_rows, num_rows, 19.5)
        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
        aruco_params = cv2.aruco.DetectorParameters()

        ######## Loop Utama ############
        
        while not stop_event.is_set():
            current_time = perf_counter()

            # if (current_time - last_time_input) >= waktu_input:
            # terus ambil gambar dari kamera
            ret, frame = camera.read()
            if not ret:
                print("Error: Tidak dapat membaca frame dari kamera.")
                stop_event.set()
                break
            
            # deteksi marker dari frame
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

            # pemrosesan bila terdeteksi aruco
            if ids is not None:
                x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah = convertPos(frame, ids, corners, dataID, databasePos)
                xAvg, yAvg, dataMovAvgX, dataMovAvgY = movAvg(window, dataMovAvgX, dataMovAvgY, xEst, yEst)
                plotX.append(xAvg)
                plotY.append(yAvg)
                posisiX[0] = xAvg
                posisiY[0] = yAvg
                #save data
                dataAkhir.append([current_time, xAvg, yAvg, Vforward, Vbackward, Vleft, Vright])
                if (len(id_1) != len(ids)) or ((len(id_1) == len(ids)) and (id_1[0] != ids[0])):
                    # print("ArUco markers detected. IDs: ", ids)
                    id_1 = ids

            # menampilkan tangkapan kamera
            cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)
            
            # Indikasikan bahwa video stream telah berhasil dibuka
           
            if kameraCheck == False:
                video_ready_event.set()  
                kameraCheck = True

            # Memastikan koneksi UDP ke drone
            if setup_udp == False:
                drone_connect_event.wait()
                setup_udp = True
                
                # last_time_input = current_time
            
            # algoritma pergerakan dan kontrol
            if (moveCommand == True) and (setup_udp == True):
                if setup_moving == False:    
                    takeoff(drone)
                    sleep(4)  # Tunggu beberapa detik untuk memastikan drone stabil
                    print("Drone Take off")
                    hover(drone)
                    setup_moving = True

            #     # # Perbarui pergerakan drone menuju target
            #     # if (((current_time - last_time_move) >= waktu_move) and (setup_moving == True)) :
            #     #     if posisiX[0] != None :
            #     #         if posisiY[0] != None :
            #     #             first, error_x1, error_y1 = move_to_target(drone, posisiX[0], posisiY[0], target_x, target_y, first, error_x1, error_y1)
            #     #             # print("x = "+str(posisiX[0])+" , y = "+str(posisiY[0]))
            #     #     last_time_move = current_time
            #     # # Cek apakah sudah mendekati target
            #     # if abs(target_x - posisiX[0]) < 15 and abs(target_y - posisiY[0]) < 15:
            #     #     print("Drone sudah sampai target, akan mendarat")
            #     #     hover(drone)
            #     #     sleep(2)
            #     #     land(drone)
            #     #     moveCommand = False
                if loop > loopWait:
                    if (((current_time - last_time_move) >= waktu_move) and (setup_moving == True)):
                        if direction == 0:
                            direction = 1
                            print("Drone Maju")
                        elif direction == 1:
                            hover(drone)
                            direction = 5
                            print("Trajektori selesai, akan mendarat")
                        last_time_move = current_time

                    if (((current_time - last_time_command) >= waktu_command_move) and (setup_moving == True)):
                        if setup_moving == True:
                            if direction == 1:
                                move(drone, forward=kec)
                            elif direction == 5:

                                land(drone)
                                print("Drone Mendarat")
                                moveCommand = False
                        last_time_command = current_time

            if cv2.waitKey(1) & 0xFF == ord('q'):
                stop_event.set()
                stopping = True
                break
            loop +=1
        camera.release()
        cv2.destroyAllWindows()

    except Exception as e:
        print(f"Error during video streaming: {e}")
    
    except KeyboardInterrupt:
        print("Program dihentikan oleh pengguna. Drone akan Mendarat")
        # land(drone)
        print("Drone mendarat")
        stop_event.set()

    udpConnect_thread.join()

    np.savetxt("Data_Akhir.txt", dataAkhir, fmt="%s")
    drone = disconnect(drone)

    import matplotlib
    matplotlib.use('TkAgg')
    import matplotlib.pyplot as plt
    plt.ion()
    figure, ax = plt.subplots(figsize=(10, 8))
    line1, = ax.plot(plotX, plotY, color='blue', marker='.', linewidth=0.5, markersize=15)
    ax.set_xlim(0, 400)
    ax.set_ylim(0, 500)
    plt.grid(True)
    ax.relim()
    ax.autoscale_view()
    figure.canvas.draw()
    figure.canvas.flush_events()
    plt.ioff()
    plt.show(block=True)

video nyala, bisa lanjut
connected
None
drone konek
takeoff
Drone Take off
drone berhasil hover
Drone Maju
drone berhasil hover
Trajektori selesai, akan mendarat
landing
Drone Mendarat


: 

<h2>Rekomendasi dari chatGPT</h2>

In [ ]:
import logging
# Setup logging
logging.basicConfig(level=logging.INFO)

class DroneControl:
    def __init__(self, ip):
        self.ip = ip
        self.drone = None
        self.connected = False
        self.navdemo = None
        self.moveCommand = False
        self.setup_udp = False
        self.setup_moving = False
        self.direction = 0
        self.kec = 0.1
        self.target_x = 200
        self.target_y = 250
        self.loopWait = 250
        self.loop = 0
        self.kameraCheck = False
        self.first = True
        self.error_x1 = 0
        self.error_y1 = 0
        self.video_ready_event = threading.Event()
        self.drone_connect_event = threading.Event()
        self.stop_event = threading.Event()
        
        self.init_time = perf_counter()
        self.last_time_move = self.init_time
        self.last_time_command = self.init_time
        self.waktu_move = 4
        self.waktu_command_move = 0.03
        
        self.posisiX = [None]
        self.posisiY = [None]
        self.dataAkhir = [["Waktu", "X_movAvg", "y_movAvg", "Pusat_x", "Pusat_y", "MFP_X", "MFP_Y", "Pixel_pusat_X", "Pixel_pusat_Y", "Pixel_MFP_X", "Pixel_MFP_Y"]]
        self.dataMovAvgX = []
        self.dataMovAvgY = []

    def udpConnect(self):
        try:
            # Tunggu hingga video stream siap
            self.video_ready_event.wait()
            logging.info("Video nyala, bisa lanjut")
            self.drone, self.navdemo, self.connected = connect(self.drone, self.ip, self.navdemo)
            if self.drone is not None and self.connected:
                logging.info("Drone terkoneksi")
                self.drone.navdata_ready.wait()  # wait until NavData is ready
                self.moveCommand = True
                self.drone_connect_event.set()
            else:
                logging.warning("Tidak dapat koneksi dengan drone")
        except Exception as e:
            logging.error(f"Error communicating via UDP: {e}")

    def main_loop(self):
        try:
            ardrone_url = 'tcp://192.168.1.1:5555'
            camera = cv2.VideoCapture(ardrone_url)
            
            if not camera.isOpened():
                logging.error("Gagal membuka stream video. Pastikan URL benar dan drone terhubung.")
                self.stop_event.set()
                return

            dataID, databasePos = dataAruco(num_rows, num_rows, 19.5)
            aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000)
            aruco_params = cv2.aruco.DetectorParameters()

            while not self.stop_event.is_set():
                current_time = perf_counter()
                ret, frame = camera.read()
                if not ret:
                    logging.error("Error: Tidak dapat membaca frame dari kamera.")
                    self.stop_event.set()
                    break
                
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)
                
                if ids is not None:
                    x, y, posPixelX, posPixelY, xEst, yEst, pusatKameraX, pusatKameraY, tengah = convertPos(frame, ids, corners, dataID, databasePos)
                    xAvg, yAvg, self.dataMovAvgX, self.dataMovAvgY = movAvg(window, self.dataMovAvgX, self.dataMovAvgY, xEst, yEst)
                    self.posisiX[0] = xAvg
                    self.posisiY[0] = yAvg
                    self.dataAkhir.append([current_time, xAvg, yAvg, xEst, yEst, x, y, pusatKameraX, pusatKameraY, posPixelX, posPixelY])
                
                cameraDisplay(frame, corners, ids, posPixelX, posPixelY, pusatKameraX, pusatKameraY, tengah)
                
                if not self.kameraCheck:
                    self.video_ready_event.set()
                    self.kameraCheck = True

                if not self.setup_udp:
                    self.drone_connect_event.wait()
                    self.setup_udp = True
                
                if self.moveCommand and self.setup_udp:
                    if not self.setup_moving:    
                        takeoff(self.drone)
                        sleep(4)
                        logging.info("Drone Take off")
                        hover(self.drone)
                        self.setup_moving = True

                    if self.loop > self.loopWait:
                        if (current_time - self.last_time_move) >= self.waktu_move:
                            if self.direction == 0:
                                self.direction = 1
                                logging.info("Drone Maju")
                                string = "maju"
                                self.dataAkhir.append([current_time, -1, -1, string, string,string, string, string, string, string, string])
                            elif self.direction == 1:
                                hover(self.drone)
                                self.direction = 5
                                logging.info("Trajektori selesai, akan mendarat")
                                string = "landing"
                                self.dataAkhir.append([current_time, -1, -1, string, string,string, string, string, string, string, string])
                            self.last_time_move = current_time

                        if (current_time - self.last_time_command) >= self.waktu_command_move:
                            if self.setup_moving:
                                if self.direction == 1:
                                    move(self.drone, forward=self.kec)
                                elif self.direction == 5:
                                    land(self.drone)
                                    logging.info("Drone Mendarat")
                                    self.moveCommand = False
                            self.last_time_command = current_time

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    self.stop_event.set()
                    break
                self.loop += 1

            camera.release()
            cv2.destroyAllWindows()
        except Exception as e:
            logging.error(f"Error during video streaming: {e}")
        except KeyboardInterrupt:
            logging.info("Program dihentikan oleh pengguna. Drone akan Mendarat")
            land(self.drone)
            self.stop_event.set()

    def run(self):
        udpConnect_thread = threading.Thread(target=self.udpConnect)
        udpConnect_thread.start()
        self.main_loop()
        udpConnect_thread.join()
        np.savetxt("Data_Akhir.txt", self.dataAkhir, fmt="%s")
        self.drone = disconnect(self.drone)

if __name__ == "__main__":
    drone_control = DroneControl(ip='192.168.1.1')
    drone_control.run()
